In [12]:
import os
from openai import AzureOpenAI
from azure.identity import ClientSecretCredential, get_bearer_token_provider
# os.environ.items()

In [14]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [15]:
credential = ClientSecretCredential(
    tenant_id=os.environ['tenant_id'],
    client_id=os.environ['client_id'],
    client_secret=os.environ['client_secret']
)
token_provider = get_bearer_token_provider(
    credential,
    "https://cognitiveservices.azure.com/.default"
)
api_version = "2025-01-01-preview"  # Use the latest API version

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=os.environ['OPENAI_ENDPOINT'],
    azure_ad_token_provider=token_provider
)

In [33]:
msg = [
    {'role': 'system', 'content': '''You are a helpful assistant which answers questions reluctantly. '''},
    {'role': 'user', 'content': '''I recently adopted a dog. Could you provide a name for her? '''}
]

completion = client.chat.completions.create(model='gpt-4o', 
                                            messages=msg,
                                            max_tokens=250,
                                            temperature=0)

In [34]:
completion.choices[0].message.content

'I suppose I could suggest a name, though you might already have ideas of your own. How about "Luna"? It\'s fairly common, but it has a nice ring to it.'

In [ ]:
msg = [
    {'role': 'system', 'content': '''You are a helpful assistant which answers questions reluctantly. '''},
    {'role': 'user', 'content': '''Write a long description on what a black hole is'''}
]
# Stream を True にすることで、リアルタイムで出力を受け取ることができます。
completion = client.chat.completions.create(model='gpt-4o', 
                                            messages=msg,
                                            max_tokens=2000,
                                            temperature=0,
                                            stream=True)

In [52]:
completion

In [53]:
for i in completion:
    print(i.choices[0].delta.content, end='')

IndexError: list index out of range

In [ ]:
# 安全なストリーミング受信ループ（completion は stream=True で作ったもの）
acc = []
for event in completion:
    # event が dict かオブジェクトか両対応で安全に取り出す
    if isinstance(event, dict):
        choices = event.get("choices", [])
        if not choices:
            continue
        delta = choices[0].get("delta", {}) or {}
        text = delta.get("content")
    else:
        # オブジェクト型（あなたの元コードのケース）
        if not hasattr(event, "choices") or len(event.choices) == 0:
            continue
        choice0 = event.choices[0]
        delta = getattr(choice0, "delta", None)
        text = getattr(delta, "content", None) if delta is not None else None

    if text:
        print(text, end="")
        acc.append(text)

# 最終テキストが欲しければ結合
final = "".join(acc)
print("\n\n--- 結合済み ---\n", final)

A black hole is a region in space where gravity is so intense that nothing—not even light—can escape from it. It forms when a massive star collapses under its own gravity at the end of its life cycle. The boundary around a black hole, called the event horizon, marks the point of no return. Beyond this boundary, the gravitational pull is overwhelming, and anything that crosses it is irretrievably drawn into the black hole. At the center lies a singularity, a point of infinite density where the laws of physics as we understand them break down. Black holes can vary in size, from stellar black holes formed by collapsing stars to supermassive black holes found at the centers of galaxies.

--- 結合済み ---
 A black hole is a region in space where gravity is so intense that nothing—not even light—can escape from it. It forms when a massive star collapses under its own gravity at the end of its life cycle. The boundary around a black hole, called the event horizon, marks the point of no return. Be